In [34]:
import pandas as pd
import seaborn as sns
import datetime
from datetime import *
import sys
import gzip
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pylab
import scipy.stats as stats

In [35]:
plt.style.use('seaborn')
pd.options.mode.chained_assignment = None  # default='warn'

# beer

In [36]:
#load the data beers.csv
df_beers = pd.read_csv('./data/beers.csv')

In [37]:
df_beers.head()

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.0,NaN,NaN,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.5,NaN,NaN,0,NaN


In [38]:
list_considered_variable=['beer_id','beer_name','brewery_id','brewery_name','style',
                          'nbr_reviews','nbr_ratings','avg', 'abv']
df_beers=df_beers[list_considered_variable]

In [39]:
beer_catalog = {
    'Bocks' : ['Bock', 'Doppelbock', 'Eisbock', 'Maibock', 'Weizenbock', 'Maibock / Helles Bock'],

    'Dark Ales' : ['Dubbel', 'Roggenbier', 'Scottish Ale', 'Winter Warmer'],

    'Hybrid Beers' : ['Bière de Champagne / Bière Brut', 'Braggot', 'California Common / Steam Beer', 'Cream Ale'],

    'Pale Ales' : ['American Amber / Red Ale', 'American Blonde Ale', 'American Pale Ale', 'Belgian Blonde Ale',
    'Belgian Pale Ale', 'Bière de Garde', 'English Bitter', 'English Pale Ale', 'English Pale Mild Ale', 'Extra Special / Strong Bitter (ESB)',
    'Grisette', 'Irish Red Ale', 'Kölsch', 'Saison', 'Saison / Farmhouse Ale', 'Belgian Strong Pale Ale', 'American Pale Ale (APA)'],

    'Porters' : ['American Porter', 'Baltic Porter', 'English Porter', 'Imperial Porter', 'Robust Porter ', 'Smoked Porter'],

    'Stouts' : ['American Imperial Stout', 'American Stout', 'English Stout', 'Foreign / Export Stout', 'Irish Dry Stout', 'Oatmeal Stout',
    'Russian Imperial Stout', 'Milk / Sweet Stout', 'American Double / Imperial Stout'],

    'Wheat Beers' : ['American Dark Wheat Beer', 'American Pale Wheat Beer', 'Dunkelweizen', 'Grodziskie', 'Hefeweizen', 'Kristallweizen', 'Witbier', 
    'American Pale Wheat Ale', 'American Dark Wheat Ale', 'Berliner Weissbier', 'Kristalweizen'],

    'Brown Ales' : ['Altbier', 'American Brown Ale', 'Belgian Dark Ale', 'English Brown Ale', 'English Dark Mild Ale', 'Belgian Strong Dark Ale'],

    'Dark Lagers' : ['American Amber / Red Lager', 'Czech Amber Lager', 'Czech Dark Lager', 'European Dark Lager', 'Märzen',
    'Munich Dunkel', 'Rauchbier', 'Schwarzbier', 'Vienna Lager', 'Munich Dunkel Lager', 'Euro Dark Lager'],

    'India Pale Ales' : ['American IPA', 'American Double / Imperial IPA', 'Belgian IPA', 'Black IPA', 'Brut IPA', 'English IPA', 'English India Pale Ale (IPA)', 
    'Imperial IPA', 'Milkshake IPA', 'New England IPA', 'American Black Ale'],

    'Pale Lagers' : ['American Adjunct Lager', 'American Lager', 'Bohemian / Czech Pilsner', 'Czech Pale Lager', 'European / Dortmunder Export Lager',
    'European Pale Lager', 'European Strong Lager', 'Festbier / Wiesnbier', 'German Pilsner', 'Helles', 'Imperial Pilsner', 'India Pale Lager (IPL)',
    'Kellerbier / Zwickelbier', 'Light Lager', 'Malt Liquor', 'Czech Pilsener', 'Euro Pale Lager', 'American Pale Lager', 'German Pilsener', 'Dortmunder / Export Lager', 
    'Euro Strong Lager', 'Munich Helles Lager', 'American Double / Imperial Pilsner', 'American Malt Liquor', 'Märzen / Oktoberfest'],

    'Specialty Beer' : ['Chile Beer', 'Fruit and Field Beer', 'Gruit / Ancient Herbed Ale', 'Happoshu', 'Herb and Spice Beer', 'Japanese Rice Lager',
    'Kvass', 'Low-Alcohol Beer', 'Pumpkin Beer', 'Rye Beer', 'Sahti', 'Smoked Beer', 'Herbed / Spiced Beer', 'Pumpkin Ale', 'Scottish Gruit / Ancient Herbed Ale', 'Low Alcohol Beer'],

    'Strong Ales' : ['American Barleywine', 'American Strong Ale', 'Belgian Dark Strong Ale', 'Belgian Pale Strong Ale', 'English Barleywine', 'English Strong Ale',
    'Imperial Red Ale', 'Old Ale', 'Quadrupel (Quad)', 'Scotch Ale / Wee Heavy', 'Tripel', 'Wheatwine', 'Fruit / Vegetable Beer'],
    
    'Wild/Sour Beers' : ['Berliner Weisse', 'Brett Beer', 'Faro', 'Flanders Oud Bruin', 'Flanders Red Ale', 'Fruit Lambic',
    'Fruited Kettle Sour', 'Gose', 'Gueuze', 'Lambic', 'Wild Ale', 'Lambic - Unblended', 'Lambic - Fruit', 'American Wild Ale'],

    'Weird Cocktail' : ['Black & Tan']
}

In [40]:
beer_catalog_reverse = {}
for key, values in beer_catalog.items():
    for value in values:
            beer_catalog_reverse.setdefault(value, []).append(key)

In [41]:
def category_check(beer_style, beer_catalog):
    '''
    Return the new columns of 15 categories made in the website https://www.beeradvocate.com/beer/styles/
    '''
    for category_name, beer_styles in beer_catalog.items():
        if beer_style in beer_styles:
            return category_name

In [42]:
df_beers['category'] = df_beers['style'].apply(lambda beer_style: category_check(beer_style, beer_catalog))

In [43]:
df_beers

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_reviews,nbr_ratings,avg,abv,category
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.7,Pale Lagers
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,3.8,Pale Lagers
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.8,Pale Lagers
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.0,Pale Lagers
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,4.5,Pale Lagers
...,...,...,...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,3,2.71,5.0,Pale Ales
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,1,2,3.10,5.3,Brown Ales
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,0,NaN,NaN,Pale Lagers
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,0,NaN,7.0,Strong Ales


# user

In [44]:
#load the data
df_users = pd.read_csv('./data/users.csv')
print(df_users.describe())
df_users.head(5)

         nbr_ratings    nbr_reviews        joined
count  153704.000000  153704.000000  1.510520e+05
mean       54.605163      16.847876  1.357238e+09
std       252.388790     139.846706  9.195131e+07
min         1.000000       0.000000  8.407944e+08
25%         1.000000       0.000000  1.303121e+09
50%         3.000000       0.000000  1.391944e+09
75%        16.000000       2.000000  1.417691e+09
max     12046.000000    8970.000000  1.501495e+09


,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


In [152]:
#keep user from the USA only
df_us_users = df_users[df_users.location.str.contains('United States', na=False)]

#choose a minimum number of ratings per user, since we want to analyse an evolution
cutoff_low = 500 #subjective, good so ?
print("Cutoff low : ", cutoff_low)

#choose a maximum number of ratings per user, we want to filter our super-users or organisations
cutoff_high = df_us_users.nbr_ratings.quantile(0.99)
print("Cutoff high : ", cutoff_high)

#create a valid user dataframe and a set with the valid user_names
df_valid_us_users = df_us_users[(df_us_users.nbr_ratings < cutoff_high) &
                                (df_us_users.nbr_ratings >= cutoff_low)]
valid_us_users_names = set(df_valid_us_users.user_name)
print("Number of users left : ", len(valid_us_users_names))
print("Total of ratings left : ", df_valid_us_users.nbr_ratings.sum())

Cutoff low :  500
Cutoff high :  1135.0
Number of users left :  2232
Total of ratings left :  1652164


In [153]:
df_valid_us_users

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
8,1131,188,okcnittany.144868,OKCNittany,1.182766e+09,"United States, Oklahoma"
15,732,14,byofb.537281,BYOFB,1.292065e+09,"United States, Wisconsin"
17,585,389,ferrari29.320392,ferrari29,1.239790e+09,"United States, Illinois"
33,1078,10,colsanders123.545441,ColSanders123,1.293620e+09,"United States, Georgia"
35,914,890,jays2629.517433,JayS2629,1.287828e+09,"United States, Alabama"
...,...,...,...,...,...,...
43556,548,21,crafteyedoc.675112,CraftEyeDoc,1.336039e+09,"United States, North Carolina"
43626,630,23,pmoney.592770,pmoney,1.302862e+09,"United States, Illinois"
44819,637,48,aleckpa.541457,aleckpa,1.292843e+09,"United States, North Carolina"
52196,1117,4,dinoblueshoes.607196,DinoBlueShoes,1.309342e+09,"United States, Maine"


# ratings


In [154]:
def parse(filename):
    """
    Parse a txt.gz file and return a generator for it
    Copyright © 2017 Gael Lederrey <gael.lederrey@epfl.ch>
    :param filename: name of the file
    :return: Generator to go through the file
    """
    file = gzip.open(filename, 'rb')
    entry = {}
    # Go through all the lines
    for line in file:
        # Transform the string-bytes into a string
        line = line.decode("utf-8").strip()

        # We check for a colon in each line
        colon_pos = line.find(":")
        if colon_pos == -1:
            # if no, we yield the entry
            yield entry
            entry = {}
            continue
        # otherwise, we add the key-value pair to the entry
        key = line[:colon_pos]
        value = line[colon_pos + 2:]
        entry[key] = value

In [155]:
gen = parse('./data/ratings.txt.gz')

In [156]:
create = False

if create:
    undesired_features = set(['text','review', 'brewery_name', 'brewery_id', 'user_id', 'beer_name'])
    full_ratings_ls = []
    int_cols = ['beer_id']
    float_cols = ['abv','appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']

    review = next(gen)
    cols = [key for key, _ in review.items() if key not in undesired_features]

    for idx, review in enumerate(gen):
        user_name = review['user_name']
        valid_rows = [value for key, value in review.items() if key not in undesired_features]
        if user_name in valid_us_users_names:
            full_ratings_ls.append(valid_rows)

    df_ratings = pd.DataFrame(full_ratings_ls, columns = cols)

    df_ratings[int_cols] = df_ratings[int_cols].astype('int')
    df_ratings[float_cols] = df_ratings[float_cols].astype('float') 
    df_ratings["date"] = pd.to_datetime(df_ratings.date.apply(
        lambda date : datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d')
    ))

    #There is one review in 2000, wich is an outlier
    start_date = pd.to_datetime("2001-01-01")
    df_ratings = df_ratings.loc[df_ratings['date'] >= start_date]   
    df_ratings.to_pickle('./data/ratings_valid_user.pkl')

else:
    df_ratings = pd.read_pickle('./data/ratings_valid_user.pkl')

In [157]:
#add the category of the beer
beers_id_cat = df_beers[['beer_id', 'category']]
df_ratings_cat = pd.merge(df_ratings, beers_id_cat, on = 'beer_id')

In [158]:
df_ratings_cat

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category
0,19827,English Pale Ale,4.8,2014-02-10,OKCNittany,NaN,NaN,NaN,NaN,NaN,3.75,Pale Ales
1,19827,English Pale Ale,4.8,2012-12-02,BYOFB,NaN,NaN,NaN,NaN,NaN,3.50,Pale Ales
2,19827,English Pale Ale,4.8,2012-11-17,ferrari29,3.0,3.5,2.5,2.5,2.5,2.77,Pale Ales
3,19827,English Pale Ale,4.8,2011-07-16,ColSanders123,4.0,3.5,3.0,3.5,3.5,3.48,Pale Ales
4,19827,English Pale Ale,4.8,2011-07-11,JayS2629,3.0,3.5,2.5,3.0,3.0,3.07,Pale Ales
...,...,...,...,...,...,...,...,...,...,...,...,...
1652159,19149,Irish Dry Stout,6.5,2006-02-19,Dithyramb,4.0,3.0,2.0,3.0,3.0,2.96,Stouts
1652160,19142,English India Pale Ale (IPA),5.6,2006-02-19,Dithyramb,3.0,3.0,3.0,2.0,3.0,2.60,India Pale Ales
1652161,19141,American Pale Ale (APA),6.8,2006-02-19,Dithyramb,4.0,4.0,4.0,4.0,4.0,4.00,Pale Ales
1652162,19139,Kölsch,5.0,2006-02-19,Dithyramb,3.0,3.0,3.0,2.5,3.0,2.80,Pale Ales


In [159]:
df_ratings_cat.sort_values(by = "date", inplace=True)
df_ratings_cat["rating_number"] = df_ratings_cat.groupby("user_name").transform("cumcount")

In [160]:
df_filt = df_ratings_cat[df_ratings_cat["rating_number"] < 500]

In [161]:
df_filt

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number
1640018,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0
311428,3558,Czech Pilsener,5.00,2001-06-18,Mark,4.00,4.00,3.00,4.00,4.00,3.90,Pale Lagers,0
17313,73,Milk / Sweet Stout,5.20,2001-06-18,Mark,5.00,5.00,5.00,5.00,5.00,5.00,Stouts,1
136697,261,Vienna Lager,4.48,2001-06-24,Mark,4.00,3.00,3.00,3.00,3.00,3.06,Dark Lagers,2
442671,87,American Pale Ale (APA),5.40,2001-06-24,Mark,4.00,4.00,1.00,3.00,3.00,3.10,Pale Ales,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350352,205333,Belgian Pale Ale,5.50,2017-07-26,Blackop555,4.00,3.25,4.00,3.25,3.50,3.42,Pale Ales,495
976033,133247,American IPA,6.50,2017-07-26,Blackop555,3.75,4.25,3.75,4.00,4.00,4.02,India Pale Ales,496
1354178,11947,Fruit / Vegetable Beer,5.50,2017-07-27,Blackop555,3.75,3.50,3.25,3.25,3.25,3.34,Strong Ales,497
1358394,272569,Witbier,4.80,2017-07-27,Blackop555,3.75,4.00,3.75,3.50,3.75,3.71,Wheat Beers,498


In [281]:
#create a column number of days active
number_of_day_active=df_filt.groupby(['user_name']).date.unique()
number_of_day_active.name='Number'
number_of_day_active=pd.DataFrame(number_of_day_active)
number_of_day_active['Number of day active']=number_of_day_active.apply(lambda g : len(g.Number),axis=1)
del number_of_day_active['Number']
df_number_active = df_filt.merge(number_of_day_active,right_on='user_name', left_on = 'user_name')
df_number_active = df_number_active[df_number_active["Number of day active"] >= 100]

In [282]:
df_number_active

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active
0,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0,178
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.50,4.00,3.00,4.00,4.00,3.93,Pale Ales,1,178
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.50,4.50,4.00,3.50,4.00,3.95,Stouts,2,178
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.50,3.00,3.00,2.50,2.50,2.73,Pale Lagers,3,178
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.50,4.50,4.50,4.50,3.50,4.30,Brown Ales,4,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115495,205333,Belgian Pale Ale,5.5,2017-07-26,Blackop555,4.00,3.25,4.00,3.25,3.50,3.42,Pale Ales,495,170
1115496,133247,American IPA,6.5,2017-07-26,Blackop555,3.75,4.25,3.75,4.00,4.00,4.02,India Pale Ales,496,170
1115497,11947,Fruit / Vegetable Beer,5.5,2017-07-27,Blackop555,3.75,3.50,3.25,3.25,3.25,3.34,Strong Ales,497,170
1115498,272569,Witbier,4.8,2017-07-27,Blackop555,3.75,4.00,3.75,3.50,3.75,3.71,Wheat Beers,498,170


In [283]:
#Difference in days between fisrt review and last review
diff_rating_per_user=df_number_active.groupby(['user_name']).apply(lambda group : (pd.Series(group.date.max()-group.date.min()).dt.days))

diff_rating_per_user.name='Different days'
diff_rating_per_user=pd.DataFrame(diff_rating_per_user)

df_diff_days = df_number_active.merge(diff_rating_per_user,right_on='user_name', left_on = 'user_name')

#df_diff_days = df_diff_days[df_diff_days["Different days"] > pd.Timestamp(365)]
#df_diff_days.rename = ["0"]
df_diff_days = df_diff_days.rename(columns = {0 : "Different days"})
df_diff_days = df_diff_days[df_diff_days["Different days"] > 365]
df_diff_days


,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active,Different days
0,167,Witbier,NaN,2001-03-16,frank4sail,4.50,4.50,4.00,5.00,5.00,4.75,Wheat Beers,0,178,1310
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.50,4.00,3.00,4.00,4.00,3.93,Pale Ales,1,178,1310
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.50,4.50,4.00,3.50,4.00,3.95,Stouts,2,178,1310
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.50,3.00,3.00,2.50,2.50,2.73,Pale Lagers,3,178,1310
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.50,4.50,4.50,4.50,3.50,4.30,Brown Ales,4,178,1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890495,289531,Altbier,6.5,2017-07-01,Jonathan-Morgan,3.50,3.50,3.50,3.50,3.50,3.50,Brown Ales,495,107,376
890496,289527,American Brown Ale,7.3,2017-07-01,Jonathan-Morgan,3.50,3.75,3.50,3.75,3.50,3.66,Brown Ales,496,107,376
890497,289533,Extra Special / Strong Bitter (ESB),5.0,2017-07-01,Jonathan-Morgan,3.50,3.50,3.50,3.50,3.50,3.50,Pale Ales,497,107,376
890498,289562,American Pale Ale (APA),6.5,2017-07-01,Jonathan-Morgan,3.50,3.75,3.50,3.75,3.50,3.66,Pale Ales,498,107,376


In [284]:
x = df_diff_days.groupby(["user_name", pd.Grouper(freq = "D", key="date")])["beer_id"].count()
x.describe(percentiles=[0.99])

count    383426.000000
mean          2.156870
std           3.929989
min           1.000000
50%           1.000000
99%          13.000000
max         314.000000
Name: beer_id, dtype: float64

In [285]:
x = df_diff_days.groupby(["user_name", pd.Grouper(freq = "M", key="date")], as_index=False)["beer_id"].count()
y = x.groupby("user_name")["beer_id"].max()
#x[x["user_name"] == "-Rick"]
y.describe()

count    1654.000000
mean       69.998791
std        45.606619
min        14.000000
25%        40.000000
50%        56.000000
75%        84.000000
max       389.000000
Name: beer_id, dtype: float64

In [286]:
valid = y[(y <= 70)]
df_max_per_mounth = df_diff_days[df_diff_days["user_name"].isin(set(valid.index))]

In [287]:
df_reviews_final = df_max_per_mounth.copy()

In [288]:
df_reviews_final

,beer_id,style,abv,date,user_name,appearance,aroma,palate,taste,overall,rating,category,rating_number,Number of day active,Different days
0,167,Witbier,NaN,2001-03-16,frank4sail,4.5,4.5,4.0,5.00,5.0,4.75,Wheat Beers,0,178,1310
1,1550,American Pale Ale (APA),5.0,2002-03-26,frank4sail,4.5,4.0,3.0,4.00,4.0,3.93,Pale Ales,1,178,1310
2,19,Milk / Sweet Stout,4.9,2002-03-26,frank4sail,4.5,4.5,4.0,3.50,4.0,3.95,Stouts,2,178,1310
3,102,Märzen / Oktoberfest,5.3,2002-03-26,frank4sail,3.5,3.0,3.0,2.50,2.5,2.73,Pale Lagers,3,178,1310
4,3089,Belgian Strong Dark Ale,18.0,2002-03-26,frank4sail,4.5,4.5,4.5,4.50,3.5,4.30,Brown Ales,4,178,1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888995,260713,American Double / Imperial Stout,13.0,2017-05-23,JLK7299,5.0,4.5,4.5,4.50,4.5,4.53,Stouts,495,206,401
888996,262165,American Strong Ale,12.5,2017-05-28,JLK7299,5.0,5.0,4.5,4.50,4.5,4.65,Strong Ales,496,206,401
888997,282574,American IPA,6.8,2017-05-28,JLK7299,4.0,4.0,4.0,4.00,4.0,4.00,India Pale Ales,497,206,401
888998,283390,American IPA,7.2,2017-05-28,JLK7299,5.0,5.0,4.5,4.25,4.5,4.55,India Pale Ales,498,206,401


In [289]:
def compute_diff_date(user):
    diff  = pd.DataFrame()
    for day in [25, 50, 75, 100, 125, 150, 500]:
        diff[str(day)] = pd.Series(user[user["rating_number"] == day-1]["date"].values -\
                         user[user["rating_number"] == 0]["date"].values).dt.days
    return diff

In [290]:
df_reviews_final["date"] = pd.to_datetime(df_reviews_final["date"])

In [291]:
grouped = df_reviews_final.groupby("user_name").apply(compute_diff_date).reset_index()
del grouped["level_1"]

In [292]:
#grouped = grouped[grouped["150"] < 2*365]

In [293]:
grouped["user_name"] = grouped.index
grouped_melt = pd.melt(grouped, id_vars=["user_name"], value_vars=["25", "50", "75", "100", "125", "150", "500"])
grouped_melt.columns = ["user_name", "cat_nb_days","day_diff"]
grouped_melt

,user_name,cat_nb_days,day_diff
0,0,25,0
1,1,25,32
2,2,25,23
3,3,25,49
4,4,25,11
...,...,...,...
7737,1101,500,688
7738,1102,500,1418
7739,1103,500,617
7740,1104,500,1000


In [294]:
fig = px.box(grouped_melt, x="cat_nb_days", y="day_diff")
fig.show()
fig.write_html("day_diff.html")

### cluster

In [296]:
df_to_cluster = df_reviews_final[df_reviews_final["rating_number"] < 50]

In [297]:
user_to_cat = df_to_cluster.groupby(["user_name", "category"])["beer_id"].count()
user_to_cat = user_to_cat.unstack(level=-1)
user_to_cat.fillna(0, inplace = True)

In [298]:
from sklearn.cluster import KMeans

In [299]:
our_clustering = KMeans(4).fit(user_to_cat.values)

In [300]:
import collections
# for cluster_num in range(4)    
#     np.count_nonzero(our_clustering.labels_ == 0)
count = collections.Counter(our_clustering.labels_)
count

Counter({2: 289, 0: 474, 1: 217, 3: 126})

In [301]:
df_cluster_beer = pd.DataFrame(our_clustering.cluster_centers_.T)
#df_cluster_beer.rename(columns = user_to_cat.columns)
df_cluster_beer.index = user_to_cat.columns
df_cluster_beer["style"] = df_cluster_beer.index
df_cluster_beer = pd.melt(df_cluster_beer, id_vars=["style"], value_vars=[0,1,2,3])
df_cluster_beer.columns = ["style", "cluster", "Number of beer"]
df_cluster_beer

,style,cluster,Number of beer
0,Bocks,0,1.763713
1,Brown Ales,0,3.409283
2,Dark Ales,0,1.829114
3,Dark Lagers,0,1.801688
4,Hybrid Beers,0,0.324895
5,India Pale Ales,0,7.263713
6,Pale Ales,0,9.497890
7,Pale Lagers,0,4.324895
8,Porters,0,2.333333
9,Specialty Beer,0,1.685654


In [303]:
fig = px.line(df_cluster_beer, x="style", y="Number of beer", color="cluster", title="Centroids obtained with Kmeans (k=4) of the first 50 ratings of each user")
#fig = px.line(df_cluster_beer, x="style", y=1)
fig.show()
fig.write_html("centroid.html")